<a id='top'></a>
<a name="top"></a><!--Need for Colab-->
# Quick introduction to TensorFlow Serving

## Half Plus Two model

Using Docker, subprocess module, prebuilt Half Plus Two model, and HTTP requests-logging with TensorFlow Serving.

1. [Setup](#setup)
2. [Introduction](#2.0)
3. [Using subprocess with TensorFlow Serving](#3.0)
4. [Set up the prebuilt half_plus_two model](#4.0)
    * 4.1 [Inspect the SavedModel Signature](#4.1)
5. [HTTP Request Logging for TensorFlow Serving](#5.0)
6. [Running a minimal Docker image with TensorFlow Serving](#6.0)
7. [Subprocess to write logs to file](#7.0)
    * 7.1 [Verify logs](#7.1)
8. [RESTful APIs](#8.0)
    * 8.1 [Model status API](#8.1)
    * 8.2 [Model Metadata API](#8.2)
    * 8.3 [Predict API](#8.3)
    * 8.4 [Classify and Regress API](#8.4)
9. [Miscellaneous](#9.0)
10. [End and clean up processes](#10.0)

---
<a name="setup"></a>
# 1. Setup
<a href="#top">[back to top]</a>

In [1]:
# stdlib imports
import asyncio
import glob
import json
import os
from pathlib import Path
import pprint
import shlex
import subprocess
import sys
import time

# third party imports
import tensorflow as tf
import requests

# For debugging, provides version & hardware info
try:
    %load_ext watermark
except ImportError:
    print("Installing watermark:")
    !pip install watermark -q
    %load_ext watermark
finally:
    %watermark --python --packages requests,tensorflow
    
pp = pprint.PrettyPrinter(indent=2)

def HR():
    print("-"*40)

print("Finished loading packages..")

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 7.34.0

requests  : 2.27.1
tensorflow: 2.9.1

Finished loading packages..


---
<a id="2.0"></a><a name="2.0"></a>
# 2. Introduction
<a href="#top">[back to top]</a>

We explore these tasks here:

* Using subprocess with TensorFlow Serving.
* HTTP Request Logging for TensorFlow Serving.
* Set up the prebuilt model, half_plus_two.
* Running a minimal Docker image with TensorFlow Serving.
* Subprocess to write logs to file.

---
<a id="3.0"></a><a name="3.0"></a>
# 3. Using subprocess with TensorFlow Serving
<a href="#top">[back to top]</a>



A convenient way to use TensorFlow Serving is via Docker. We can operate the server and logging functions via Docker Commands on the CLI, but there are certain advantages (readability, maintenance, security, etc) to wrapping them in Python.

The older method of doing this involved using either os.system or os.spawn*. Here, we will instead use the newer subprocess module. This allows us to spawn new processes, connect to their input/output/error pipes, and optionally obtain their return codes. This is a safer analog to os.system().

The underlying process creation and management in `subprocess` is done by the [Popen Constructor](https://docs.python.org/3/library/subprocess.html#popen-constructor), `subprocess.Popen`. The underlying Popen interface can be used directly and offers the most flexibility. By default it results in a non-blocking call.

Once you've created the Popen instance, some options are:
* `wait()`:  to pause until the subprocess has exited.
* `poll()`: check if it's exited without pausing.
* `communicate()`: interact with process
    - Send data to stdin. 
    - Read data from stdout and stderr, until end-of-file is reached. 
    - Wait for process to terminate and set the returncode attribute. 


**Popen Constructor:**

<sup>

```bash
class subprocess.Popen(
    args, 
    bufsize=- 1, 
    executable=None, 
    stdin=None, 
    stdout=None, 
    stderr=None, 
    preexec_fn=None, 
    close_fds=True, 
    shell=False, 
    cwd=None, 
    env=None, 
    universal_newlines=None, 
    startupinfo=None, 
    creationflags=0, 
    restore_signals=True, 
    start_new_session=False, 
    pass_fds=(), 
    *, 
    group=None, 
    extra_groups=None, 
    user=None, 
    umask=- 1, 
    encoding=None, 
    errors=None, 
    text=None, 
    pipesize=- 1
)

```
<br>
</sup>
    
A convenience function built upon the underlying Popen interface is `subprocess.run`. This is a blocking call, as it waits for the command(s) to complete, then return a CompletedProcess instance.

There are older high-level APIs, existing prior to Python 3.5. The functionality provided by them has been superceded by `subprocess.Popen` and `subprocess.run`:

* `subprocess.call`
* `subprocess.check_call`
* `subprocess.check_output`


**Useful resources on subprocess:**

- https://peps.python.org/pep-0324/
- https://docs.python.org/3/whatsnew/2.4.html#pep-324-new-subprocess-module
- https://docs.python.org/3/library/subprocess.html
- https://www.bogotobogo.com/python/python_subprocess_module.php
- https://qiita.com/HidKamiya/items/e192a55371a2961ca8a4 (JP)
- https://www.programcreek.com/python/example/50/subprocess.Popen


---
<a id="4.0"></a><a name="4.0"></a>
# 4. Set up the prebuilt half_plus_two model
<a href="#top">[back to top]</a>

For this example, we will use Docker to deploy Tensorflow Serving and host the toy model **half_plus_two** that computes f(x) = (x / 2) + 2.  This model is found in the Tensorflow Serving Github repository.

* https://www.tensorflow.org/tfx/serving/tensorboard
* https://www.tensorflow.org/tfx/serving/docker
* https://github.com/tensorflow/serving/tree/master/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_two_cpu/00000123


In [2]:
if not Path('serving').is_dir():
    !git clone https://github.com/tensorflow/serving
else:
    print("Saved TensorFlow models in 'serving/tensorflow_serving/servables/tensorflow/testdata'")

Saved TensorFlow models in 'serving/tensorflow_serving/servables/tensorflow/testdata'


In [3]:
# Get absolute pathway of demo models
TESTDATA=Path("serving/tensorflow_serving/servables/tensorflow/testdata").resolve()
!du -hs {TESTDATA}

9.2M	/Users/gb/Desktop/tf_server_01/serving/tensorflow_serving/servables/tensorflow/testdata


In [4]:
# Define model_name and model_dir, used to start the Docker container
model_saved = 'saved_model_half_plus_two_cpu'
model_dir = (Path() / TESTDATA / model_saved).resolve()
model_name = 'half_plus_two'
model_newest = max(glob.glob(f"{model_dir}/*"))

<a id='4.1'></a><a name='4.1'></a>
## 4.1 Inspect the SavedModel Signature
<a href="#top">[back to top]</a>

We can use the tool `saved_model_cli` to inspect the SignatureDefs (the callable methods) of a model. This allows us to confirm the input Tensor dtype and shape matches the model.

Resources:
* https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel
* https://www.tensorflow.org/guide/saved_model
* https://blog.tensorflow.org/2021/03/a-tour-of-savedmodel-signatures.html
* https://www.tensorflow.org/tfx/tutorials/serving/rest_simple

In [54]:
print("All tag-sets in the latest SavedModel:")
HR()
!saved_model_cli show --dir {model_newest}

All tag-sets in the latest SavedModel:
----------------------------------------
The given SavedModel contains the following tag-sets:
'serve'


In [56]:
print("All available SignatureDef keys in the MetaGraphDef specified by tag-set 'serve':")
HR()
!saved_model_cli show --dir {model_newest} --tag_set serve

All available SignatureDef keys in the MetaGraphDef specified by tag-set 'serve':
----------------------------------------
The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "classify_x_to_y"
SignatureDef key: "regress_x2_to_y3"
SignatureDef key: "regress_x_to_y"
SignatureDef key: "regress_x_to_y2"
SignatureDef key: "serving_default"


In [57]:
print("All inputs and outputs TensorInfo for the specific SignatureDef 'serving_default' in the MetaGraph:")
print("Note that shape: (-1) implies 1-D shape for inference data, eg {'examples': [{...}]}")
HR()

!saved_model_cli show --dir {model_newest} \
    --tag_set serve --signature_def serving_default

All inputs and outputs TensorInfo for the specific SignatureDef 'serving_default' in the MetaGraph:
Note that shape: (-1) implies 1-D shape for inference data, eg {'examples': [{...}]}
----------------------------------------
The given SavedModel SignatureDef contains the following input(s):
  inputs['x'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: x:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['y'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: y:0
Method name is: tensorflow/serving/predict


In [58]:
# Show all available information in the SavedModel

!saved_model_cli show --dir {model_newest} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classify_x_to_y']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: unknown_rank
        name: tf_example:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y:0
  Method name is: tensorflow/serving/classify

signature_def['regress_x2_to_y3']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: x2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['outputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: y3:0
  Method name is: tensorflow/serving/regress

signature_def['regress_x_to_y']:
  The given SavedModel SignatureDef 

---
<a id="5.0"></a><a name="5.0"></a>
# 5. HTTP Request Logging for TensorFlow Serving
<a href="#top">[back to top]</a>

* The easiest way to enable logging for TensorFlow-Model-Serving is via environment variables. This is not specific to TensorFlow Serving, but general to TensorFlow.

* `TF_CPP_MIN_VLOG_LEVEL` enables logging of the main C++ backend. However, even the lowest setting of `TF_CPP_MIN_VLOG_LEVEL=1` results in too much noise.

* `TF_CPP_VMODULE` provides a way to constrain logging to specific modules or source files. The general format is `TF_CPP_VMODULE=<module_name>=1`, where the module name can be either the C++ or Python file name (without the extension).

* Here, we can activate logging individually for http_server.cc via 
`TF_CPP_VMODULE=http_server=1`. This will enable simple HTTP request logging and errors.

* To use this with Docker, we pass it as an environmental variable:  `--env TF_CPP_VMODULE=http_server=1`

---
<a name="6.0"></a>
# 6. Running a minimal Docker image with TensorFlow Serving
<a href="#top">[back to top]</a>

* This is the original Docker command-line format.

* `docker run` also pulls a docker image (or repository) from the docker registry, if it doesn't already exist locally.


This docker image features

* Port 8500 exposed for gRPC
* Port 8501 exposed for the REST API
* Optional environment variable MODEL_NAME (defaults to model)
* Optional environment variable MODEL_BASE_PATH (defaults to /models)

---

It can be easier to first to experiment and set up the Docker cli-commands first, then later wrap with subprocess.Popen in Python.

In [8]:
cmd_cli = f"""
docker run \\
--rm --tty -p 8500:8500 -p 8501:8501 \\
--name {model_name} \\
--mount type=bind,source={model_dir},target=/models/{model_name} \\
--env MODEL_NAME={model_name} \\
--env TF_CPP_VMODULE='http_server=1' \\
--detach \\
--log-driver=json-file \\
--log-opt=mode=non-blocking \\
tensorflow/serving:latest
"""

print(cmd_cli)


docker run \
--rm --tty -p 8500:8500 -p 8501:8501 \
--name half_plus_two \
--mount type=bind,source=/Users/gb/Desktop/tf_server_01/serving/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_two_cpu,target=/models/half_plus_two \
--env MODEL_NAME=half_plus_two \
--env TF_CPP_VMODULE='http_server=1' \
--detach \
--log-driver=json-file \
--log-opt=mode=non-blocking \
tensorflow/serving:latest



---
For more feedback, run these commands in different terminals before instantiating the Docker container:

```bash
$ watch docker ps
$ docker logs -f half_plus_two
```

In [9]:
def server_docker():  
        
    cmd=f"""
docker run
--rm --tty -p 8500:8500 -p 8501:8501
--name {model_name}
--mount type=bind,source={model_dir},target=/models/{model_name}
--env MODEL_NAME={model_name}
--env TF_CPP_VMODULE='http_server=1'
--detach
--log-driver=json-file
--log-opt=mode=non-blocking
tensorflow/serving:latest
"""
           
    try:
        proc = subprocess.Popen(
            shlex.split(cmd),
            stdout = subprocess.PIPE,
            stderr = subprocess.PIPE
        )
        
        # The communicate() method returns a tuple (stdoutdata, stderrdata)
        # It only reads data from stdout and stderr.
        out, err = proc.communicate()
        out = out.decode()
        err = err.decode()
        print(f"out: {(out.strip())}")
        
        if err:
            print(f"err: {err}")
                        
        sleep_time = 0.5 # Small time delay for docker instance to start up
        time.sleep(sleep_time)
        
    except subprocess.CalledProcessError as e:
        print(f"Subprocess error: {e.stderr}")
    except Exception as e:
        print(f"Error: {e}")
        
    return proc 

In [10]:
# Instantiate TFS with our model 
proc = server_docker()

out: caba9295feb2aa2c568cd7f9d163dc0dc93ee91bbbc9110488fb3735da08d75d


In [11]:
# If need to immediately kill and remove unused containers/networks/images
# !docker kill {model_name} && docker system prune --force

---
<a name="7.0"></a>
# 7. Subprocess to write logs to file
<a href="#top">[back to top]</a>

We can always access logs via `docker logs <container name>` on the CLI:

In [12]:
!docker logs --tail 5 {model_name}

2022-08-07 23:49:17.507844: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2022-08-07 23:49:17.512011: I tensorflow_serving/model_servers/server.cc:393] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] getaddrinfo: address family for nodename not supported
2022-08-07 23:49:17.514273: I tensorflow_serving/model_servers/server.cc:414] Exporting HTTP/REST API at:localhost:8501 ...
[evhttp_server.cc : 245] NET_LOG: Entering the event loop ...


---
We can also create a subprocess that redirects the logs to a file:

In [13]:
def log_docker_process(log_file):  
    cmd = f"docker logs --follow {model_name}"   
    try:
        proc = subprocess.Popen(
            shlex.split(cmd),
            stdout=open(log_file, 'w'),
            stderr=subprocess.STDOUT, # redirect to stdout
        )
    except Exception as e:
        print(f"Error: {e}")

In [14]:
log_file = f"{model_name}_tfs.log"
print(f"log_file: {log_file}")

log_docker_process(log_file)

!du -hs {log_file}

log_file: half_plus_two_tfs.log
  0B	half_plus_two_tfs.log


<a id='7.1'></a><a name='7.1'></a>
## 7.1 Verify logs
<a href="#top">[back to top]</a>

Check that requests are being logged to the log file via the subprocess `log_docker_process`.

In [139]:
for x in range(10):
    resp = request_status_rest()
    print(f"===> TFS Status : {resp.headers['Date']} {resp}")
    time.sleep(0.2)
    
HR()
    
!du -h {log_file}
HR()
!tail -10 {log_file}

===> TFS Status : Mon, 08 Aug 2022 01:26:09 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:09 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:09 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:10 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:10 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:10 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:10 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:10 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:11 GMT <Response [200]>
===> TFS Status : Mon, 08 Aug 2022 01:26:11 GMT <Response [200]>
----------------------------------------
 44K	half_plus_two_tfs.log
----------------------------------------
2022-08-08 01:26:09.427568: I tensorflow_serving/model_servers/http_server.cc:162] Processing HTTP request: GET /v1/models/half_plus_two body: 0 bytes.
2022-08-08 01:26:09.646680: I tensorflow_serving/model_servers/http_server.cc:162] Proces

---
<a name="8.0"></a>
# 8. RESTful APIs
<a href="#top">[back to top]</a>

Send requests with TensorFlow ModelServer RESTful APIs

Reference:

* https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/api_rest.md
* https://github.com/tensorflow/serving/blob/master/tensorflow_serving/model_servers/tensorflow_model_server_test.py#L520


<a name="8.1"></a>
## 8.1 Model status API
<a href="#top">[back to top]</a>

* Returns the status of a model in the ModelServer.
* If successful, returns a JSON representation of `GetModelStatusResponse` protobuf.

In [124]:
try:
    resp_data = requests.get(f'http://localhost:8501/v1/models/{model_name}')
except Exception as e:
    print(f"Error: {e}")

pp.pprint(resp_data.json())

HR()
!tail -1 {log_file}

{ 'model_version_status': [ { 'state': 'AVAILABLE',
                              'status': { 'error_code': 'OK',
                                          'error_message': ''},
                              'version': '123'}]}
----------------------------------------
2022-08-08 01:20:56.911064: I tensorflow_serving/model_servers/http_server.cc:162] Processing HTTP request: GET /v1/models/half_plus_two body: 0 bytes.


<a name="8.2"></a>
## 8.2 Model Metadata API
<a href="#top">[back to top]</a>

* Returns the metadata of a model in the ModelServer. For example, we can inspect the details of the input and output tensors.
* Returns a JSON representation of `GetModelMetadataResponse` protobuf.

In [142]:
try:
    resp_data = requests.get(f'http://localhost:8501/v1/models/{model_name}/metadata')
except Exception as e:
    print(f"Error: {e}")

json_dump = (resp_data.json())
pp.pprint(repr(json_dump))

HR()

print("HTTP request log output:\n")
!tail -1 {log_file}

("{'model_spec': {'name': 'half_plus_two', 'signature_name': '', 'version': "
 "'123'}, 'metadata': {'signature_def': {'signature_def': {'regress_x_to_y': "
 "{'inputs': {'inputs': {'dtype': 'DT_STRING', 'tensor_shape': {'dim': [], "
 "'unknown_rank': True}, 'name': 'tf_example:0'}}, 'outputs': {'outputs': "
 "{'dtype': 'DT_FLOAT', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, "
 "{'size': '1', 'name': ''}], 'unknown_rank': False}, 'name': 'y:0'}}, "
 "'method_name': 'tensorflow/serving/regress'}, 'regress_x_to_y2': {'inputs': "
 "{'inputs': {'dtype': 'DT_STRING', 'tensor_shape': {'dim': [], "
 "'unknown_rank': True}, 'name': 'tf_example:0'}}, 'outputs': {'outputs': "
 "{'dtype': 'DT_FLOAT', 'tensor_shape': {'dim': [{'size': '-1', 'name': ''}, "
 "{'size': '1', 'name': ''}], 'unknown_rank': False}, 'name': 'y2:0'}}, "
 "'method_name': 'tensorflow/serving/regress'}, 'classify_x_to_y': {'inputs': "
 "{'inputs': {'dtype': 'DT_STRING', 'tensor_shape': {'dim': [], "
 "'unknown_rank':

<a name="8.3"></a>
## 8.3 Predict API
<a href="#top">[back to top]</a>

* The request body for predict API must be JSON object formatted as follows (for clarity, it may be easier to always include the "signature_name" field):

```bash
{
  // (Optional) Serving signature to use.
  // If unspecifed default serving signature is used.
  "signature_name": <string>,

  // Input Tensors in row ("instances") or columnar ("inputs") format.
  // A request can have either of them but NOT both.
  "instances": <value>|<(nested)list>|<list-of-objects>
  "inputs": <value>|<(nested)list>|<object>
}
```




* The predict request returns a JSON object in response body.

In [143]:
# Command line
!curl -d '{"signature_name": "", "instances": [1.0,2.0,5.0]}' \
    -H 'cache-control: no-cache' \
    -X POST http://localhost:8501/v1/models/half_plus_two:predict

print()
HR()

print("HTTP request log output:\n")
!tail -1 {log_file}

{
    "predictions": [2.5, 3.0, 4.5
    ]
}
----------------------------------------
HTTP request log output:

2022-08-08 01:28:07.891367: I tensorflow_serving/model_servers/http_server.cc:162] Processing HTTP request: POST /v1/models/half_plus_two:predict body: 50 bytes.


In [144]:
# In Python

# Payload
data = json.dumps({
    "instances": [1.0, 2.0, 5.0, -9]
})
print(f"Payload:\t{data}")
HR()

headers = {
    "content-type": "application_json"
}

response = requests.post(
    f'http://localhost:8501/v1/models/{model_name}:predict',
    data=data,
    headers=headers
)

print(f"Predictions:\t{json.loads(response.text)['predictions']}")
HR()

print("Properties of response:\n")
pp.pprint(response.__dict__)

HR()

print("HTTP request log output:\n")
!tail -1 {log_file}

Payload:	{"instances": [1.0, 2.0, 5.0, -9]}
----------------------------------------
Predictions:	[2.5, 3.0, 4.5, -2.5]
----------------------------------------
Properties of response:

{ '_content': b'{\n    "predictions": [2.5, 3.0, 4.5, -2.5\n    ]\n}',
  '_content_consumed': True,
  '_next': None,
  'connection': <requests.adapters.HTTPAdapter object at 0x1261ed2e0>,
  'cookies': <RequestsCookieJar[]>,
  'elapsed': datetime.timedelta(microseconds=12732),
  'encoding': 'utf-8',
  'headers': {'Content-Type': 'application/json', 'Date': 'Mon, 08 Aug 2022 01:28:14 GMT', 'Content-Length': '49'},
  'history': [],
  'raw': <urllib3.response.HTTPResponse object at 0x1261edc10>,
  'reason': 'OK',
  'request': <PreparedRequest [POST]>,
  'status_code': 200,
  'url': 'http://localhost:8501/v1/models/half_plus_two:predict'}
----------------------------------------
HTTP request log output:

2022-08-08 01:28:14.351032: I tensorflow_serving/model_servers/http_server.cc:162] Processing HTTP reques

<a name="8.4"></a>
## 8.4 Classify and Regress API
<a href="#top">[back to top]</a>

To-do

---
<a name="9.0"></a>
# 9. Miscellaneous
<a href="#top">[back to top]</a>

In [145]:
# Check environmental variables of this container
!docker exec {model_name} env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=caba9295feb2
MODEL_NAME=half_plus_two
TF_CPP_VMODULE=http_server=1
MODEL_BASE_PATH=/models
HOME=/root


In [146]:
# Using jq:
!docker inspect {model_name} | jq '.[] | .Config.Env'

[
  "MODEL_NAME=half_plus_two",
  "TF_CPP_VMODULE=http_server=1",
  "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
  "MODEL_BASE_PATH=/models"
]


In [147]:
# As a toy example, we log into the container and check this environmental variable:
!docker exec -it {model_name} /bin/bash | echo TF_CPP_VMODULE

TF_CPP_VMODULE


In [148]:
def show_tfs_dict(proc):
    print("Properties of returned container process:\n")
    for key in proc.__dict__:
        if key == 'args':
            HR()
            print(key, '->', proc.__dict__[key])
            HR()
        else:
            print(key, '->', proc.__dict__[key])
 
show_tfs_dict(proc)

Properties of returned container process:

_waitpid_lock -> <unlocked _thread.lock object at 0x126053690>
_input -> None
_communication_started -> True
----------------------------------------
args -> ['docker', 'run', '--rm', '--tty', '-p', '8500:8500', '-p', '8501:8501', '--name', 'half_plus_two', '--mount', 'type=bind,source=/Users/gb/Desktop/tf_server_01/serving/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_two_cpu,target=/models/half_plus_two', '--env', 'MODEL_NAME=half_plus_two', '--env', 'TF_CPP_VMODULE=http_server=1', '--detach', '--log-driver=json-file', '--log-opt=mode=non-blocking', 'tensorflow/serving:latest']
----------------------------------------
stdin -> None
stdout -> <_io.BufferedReader name=70>
stderr -> <_io.BufferedReader name=78>
pid -> 6242
returncode -> 0
encoding -> None
errors -> None
text_mode -> None
_sigint_wait_secs -> 0.25
_closed_child_pipe_fds -> True
_child_created -> True
_fileobj2output -> {<_io.BufferedReader name=70>: [b'c

---
<a name="10.0"></a>
# 10. End and clean up processes
<a href="#top">[back to top]</a>

In [149]:
# Kill and remove unused containers, networks, image
!docker kill {model_name} && docker system prune --force

half_plus_two
Total reclaimed space: 0B
